<a href="https://colab.research.google.com/github/berrymix13/TIL/blob/master/OneHotEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 파이썬 원-핫 인코딩(One-Hot Encoding)
- 한 개의 요소는 True, 나머지 요소는 False로 만들어주는 기법
- 원-핫 인코딩이 필요한 이유 :
    - sklearn에서 제공하는 머신러닝 알고리즘은 문자열 값을 입력값으로 허락하지 않음
    - 따라서 문자열 값은 숫자형 자료로 인코딩 하는 전처리 작업이 필요
- sklearn에서 제공하는 머신러닝 알고리즘에 필요한 데이터 구성
    - 모든 데이터는 숫자(정수형, 실수형 등)으로 구성되어있어야 한다.
    - 데이터에빈 값이 없어야 한다.

- 연습

In [ ]:
import pandas as pd

train = pd.DataFrame({'num1': [1,2,3,4,5],
                      'num2' : [10,20,30,40,50],
                      'cat1' : ['a','b','a','c','c']})
train

,num1,num2,cat1
0,1,10,a
1,2,20,b
2,3,30,a
3,4,40,c
4,5,50,c


In [ ]:
## 1. cat1의 유니크한 값 필요
cat_lst = train.cat1.unique()
print(cat_lst)

train2 = train.copy()

## (번외) cat1의 교유값에 대한 레이블 만들기
# 레이블 코딩
cat_num = [i for i, v in enumerate(cat_lst)]
print(cat_num)

## 2-1. 유니크 값과 cat1의 값이 같으면 , 다르면 0인 리스트 생성 (lst)
## 2-2. 위에서 만든 lst를 고유값들에 대한 필드로 만듦
for i in cat_lst:
    lst = []
    for j in train2.cat1:
        if j == i:
            lst.append(1)
        else:
            lst.append(0)
    train2[i] = lst
train2

['a' 'b' 'c']
[0, 1, 2]


,num1,num2,cat1,a,b,c
0,1,10,a,1,0,0
1,2,20,b,0,1,0
2,3,30,a,1,0,0
3,4,40,c,0,0,1
4,5,50,c,0,0,1


In [ ]:
# 쉬운 버전
# 데이터가 적을 때 사용함.
# prefix  : 앞 쪽에 cat1을 붙인다는 의미
train1 = pd.get_dummies(train, columns = ['cat1'], prefix = 'cat1')
train1

,num1,num2,a,b,c,cat1_a,cat1_b,cat1_c
0,1,10,1,0,0,1,0,0
1,2,20,0,1,0,0,1,0
2,3,30,1,0,0,1,0,0
3,4,40,0,0,1,0,0,1
4,5,50,0,0,1,0,0,1


In [ ]:
# df 에서 고유값들의 갯수를 세어줌
train['cat1'].value_counts()

a    2
c    2
b    1
Name: cat1, dtype: int64

In [ ]:
train['cat1'].value_counts().index

Index(['a', 'c', 'b'], dtype='object')

### 사이킷런 원핫인코딩

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# 옵션 유무에 따라 결과가 달라짐
# 무 : 위치값
ohe = OneHotEncoder()

train_cat = ohe.fit_transform(train[['cat1']])
print(train_cat)

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 0)	1.0
  (3, 2)	1.0
  (4, 2)	1.0


In [ ]:
ohe = OneHotEncoder(sparse = False)

train_cat = ohe.fit_transform(train[['cat1']])
print(train_cat)

[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [ ]:
## 레이블 값이 궁금하다면
print(ohe.categories_)
for i in ohe.categories_:
    print('cat1_'+i)

for i in ohe.categories_:
    print('cat1_'+i)

[array(['a', 'b', 'c'], dtype=object)]
['cat1_a' 'cat1_b' 'cat1_c']


In [ ]:
# columns도 컴프리헨션을 통해 계속 다르게 넣을 수 있다!!

pd.DataFrame(train_cat, columns = ['cat1_'+i for i in ohe.categories_])

,cat1_a,cat1_b,cat1_c
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0


In [ ]:
# concat을 진행할 때 drop으로 원하지 않는 데이터를 빼고 합칠 수 있다.
# drop은 기본적으로 행이 기준이라서 drop('cat1')은 오류가 나지만
# drop('cat1', axis = 1) 로 사용이 가능하다.
# drop(columns = 'cat1')
pd.concat([train.drop(columns = 'cat1'), 
           pd.DataFrame(train_cat, columns = ['cat1_'+i for i in ohe.categories_[0]])], axis = 1)
# 위에서는 멀쩡했지만, 여기서는 ohe.categories_[0]으로 해줘야 열 인덱스가 튜플구조가 아니게 된다..
# 왜?

,num1,num2,cat1_a,cat1_b,cat1_c
0,1,10,1.0,0.0,0.0
1,2,20,0.0,1.0,0.0
2,3,30,1.0,0.0,0.0
3,4,40,0.0,0.0,1.0
4,5,50,0.0,0.0,1.0


## 성적표를 이용한 원-핫 인코딩
- 원-핫 인코딩 : 분류 작업에 많이 사용함

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

In [ ]:
# 1. 데이터 읽어오기
df = pd.read_csv('성적표.csv', encoding = 'cp949')
df.head()

,순번,이름,학과,남/여,학년,이론,실기
0,1,송윤재,환경디자인원예학과,여자,2,NaN,NaN
1,2,강민형,사회복지학과,여자,3,NaN,NaN
2,3,강예린,환경디자인원예학과,여자,3,NaN,NaN
3,4,고보빈,경영학과,여자,4,NaN,NaN
4,5,김다정,보건관리학과,여자,3,NaN,NaN


In [ ]:
# 2. 이론/ 실기의 null 값을 60~100 사이의 임의 값으로 채우기
import random
df.이론 = np.random.randint(60, 101, len(df))
df.실기 = np.random.randint(60, 101, len(df))
df.head()

,순번,이름,학과,남/여,학년,이론,실기
0,1,송윤재,환경디자인원예학과,여자,2,63,87
1,2,강민형,사회복지학과,여자,3,67,74
2,3,강예린,환경디자인원예학과,여자,3,81,86
3,4,고보빈,경영학과,여자,4,85,65
4,5,김다정,보건관리학과,여자,3,87,77


In [ ]:
# 학과는 레이블 인코딩으로
# 학과를 번호와 학과에 맞춰서 고유값만 반환해줌.
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_num = le.fit_transform(df.학과) 

# print로는 list 구조이지만, 실제로는 배열구조임.
print(le_num)
print(le.classes_)

## 학과를 번호로 변경하기.
df['학과코드'] = le_num
df.head()

[16  7 16  4  6  8 14 15 14  7  9  3  4 15  4  2  4 14  0 12 10 10  6 13
  3 13  4  5 12 16  3  1 12  1 11]
['IT융합공학과' '간호학과' '건축학과' '경영정보학과' '경영학과' '동물생명자원학과' '보건관리학과' '사회복지학과'
 '상담심리학과' '식품영양학과' '영어영문학전공' '영어통번역전공' '음악학과' '일본어과' '중국어과' '화학생명과학과'
 '환경디자인원예학과']


,순번,이름,학과,남/여,학년,이론,실기,학과코드
0,1,송윤재,환경디자인원예학과,여자,2,63,87,16
1,2,강민형,사회복지학과,여자,3,67,74,7
2,3,강예린,환경디자인원예학과,여자,3,81,86,16
3,4,고보빈,경영학과,여자,4,85,65,4
4,5,김다정,보건관리학과,여자,3,87,77,6


In [ ]:
df.head()

,순번,이름,학과,남/여,학년,이론,실기,학과코드
0,1,송윤재,환경디자인원예학과,여자,2,63,87,16
1,2,강민형,사회복지학과,여자,3,67,74,7
2,3,강예린,환경디자인원예학과,여자,3,81,86,16
3,4,고보빈,경영학과,여자,4,85,65,4
4,5,김다정,보건관리학과,여자,3,87,77,6


In [ ]:
# 성별 변경
## 판다스를 이용
## 더미변수
pd.get_dummies(df['남/여']).head()
df = pd.concat([df.drop(columns='남/여'), pd.get_dummies(df['남/여'])], axis=1)
df.head()

,순번,이름,학과,학년,이론,실기,학과코드,남자,여자
0,1,송윤재,환경디자인원예학과,2,63,87,16,0,1
1,2,강민형,사회복지학과,3,67,74,7,0,1
2,3,강예린,환경디자인원예학과,3,81,86,16,0,1
3,4,고보빈,경영학과,4,85,65,4,0,1
4,5,김다정,보건관리학과,3,87,77,6,0,1


In [ ]:
pd.concat([df.drop(columns='학과'), pd.get_dummies(df.학과)], axis=1)

,순번,이름,학년,이론,실기,학과코드,남자,여자,IT융합공학과,간호학과,...,사회복지학과,상담심리학과,식품영양학과,영어영문학전공,영어통번역전공,음악학과,일본어과,중국어과,화학생명과학과,환경디자인원예학과
0,1,송윤재,2,63,87,16,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,강민형,3,67,74,7,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,강예린,3,81,86,16,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,고보빈,4,85,65,4,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,김다정,3,87,77,6,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,김두언,2,64,90,8,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,7,김민지,3,93,69,14,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
7,8,김수현,3,85,62,15,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
8,9,김영서,4,67,72,14,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
9,10,김예은,2,90,71,7,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
